# Rootzone time-series model

Time-series model that uses current rootzone conditions (EC, pH), internal greenhouse climate variables, and applied management actions (irrigation amount and fertilization type) as inputs to predict rootzone response over time. Each time step represents a single 10-minute interval aligned across all data sources.

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.base import clone
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-v0_8-whitegrid")
DATA_DIR = Path('.')
TRAIN_WINDOW = 1440  # 10 days of 10-minute samples
TEST_WINDOW = 144    # 1 day of 10-minute samples
GAP = 0            # no skip between train and test blocks
MAX_TRAIN_ROWS = None  # use full history (no cap)



In [3]:
# Load raw files
soil_path = DATA_DIR / "PH+EC Final.xlsx"
irrigation_path = DATA_DIR / "Irrigation + ALL Elemental Fractions schedule for one plant (100N).xlsx"

micro_path = DATA_DIR / "MicroClimate_predictions_all_targets_1day.csv"

soil = pd.read_excel(soil_path )
irrigation = pd.read_excel(irrigation_path)
micro = pd.read_csv(micro_path)

In [4]:
print("Soil:", soil.shape)
print("Irrigation:", irrigation.shape)
print("Micro:", micro.shape)

soil.head(), irrigation.head(), micro.head()


Soil: (111, 5)
Irrigation: (488, 11)
Micro: (14976, 10)


(   Sample        Date    PH  EC (ms)      Time
 0       1    12.06.25  6.71    0.290  11:55:00
 1       2  29.06.2025  8.20    0.290  12:08:00
 2       3    7.7.2025  7.11    0.361  11:04:00
 3       4  16.07.2025  6.75    0.387  14:00:00
 4       5  20.07.2025  6.86    0.520  12:55:00,
         DATE      TIME  Irrigation [ml] Fertilization Type  \
 0 2025-05-29  06:40:00       133.333333                NaN   
 1 2025-05-29  10:00:00       133.333333                NaN   
 2 2025-05-29  13:00:00       133.333333                NaN   
 3 2025-05-29  16:00:00       133.333333                NaN   
 4 2025-05-29  17:15:00       100.000000                  A   
 
    Ammonium Nitrate [mg] -NH4NO3  Monopotassium Phosphate[mg] -KH2PO4  \
 0                            NaN                                  NaN   
 1                            NaN                                  NaN   
 2                            NaN                                  NaN   
 3                            NaN  

In [6]:
soil["timestamp"] = pd.to_datetime(
    soil["Date"].astype(str) + " " + soil["Time"].astype(str),
    dayfirst=True
)
soil = soil.sort_values("timestamp").reset_index(drop=True)

irrigation["timestamp"] = pd.to_datetime(
    irrigation["DATE"].astype(str) + " " + irrigation["TIME"].astype(str)
)
irrigation = irrigation.sort_values("timestamp").reset_index(drop=True)


C:\Users\User\AppData\Local\Temp\ipykernel_23000\2776543669.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  soil["timestamp"] = pd.to_datetime(


In [10]:
micro["timestamp"] = pd.to_datetime(micro["timestamp"])
micro = micro.sort_values("timestamp")

In [11]:
micro_soil = pd.merge_asof(
    micro,
    soil[["timestamp", "EC (ms)", "PH"]],
    on="timestamp",
    direction="backward"
)

In [12]:
master_greenhouse_ts = pd.merge_asof(
    micro_soil,
    irrigation[["timestamp", "Irrigation [ml]", "Fertilization Type"]],
    on="timestamp",
    direction="backward",
    tolerance=pd.Timedelta("1h")  
)


In [13]:
fert_map = {
    "A": 1,
    "B": 2
}
master_greenhouse_ts["fertilizer_type"] = (
    master_greenhouse_ts["Fertilization Type"]
    .map(fert_map)
    .fillna(0)
    .astype(int)
)


master_greenhouse_ts["irrigation_ml"] = (
 master_greenhouse_ts["Irrigation [ml]"]
    .fillna(0)
)


In [19]:
print(master_greenhouse_ts[[
    "timestamp",
    "EC (ms)",
    "PH",
    "irrigation_ml",
    "fertilizer_type"
]].head(20))
master_greenhouse_ts.tail(20)

             timestamp  EC (ms)  PH  irrigation_ml  fertilizer_type
0  2025-06-08 01:00:00      NaN NaN            0.0                0
1  2025-06-08 01:10:00      NaN NaN            0.0                0
2  2025-06-08 01:20:00      NaN NaN            0.0                0
3  2025-06-08 01:30:00      NaN NaN            0.0                0
4  2025-06-08 01:40:00      NaN NaN            0.0                0
5  2025-06-08 01:50:00      NaN NaN            0.0                0
6  2025-06-08 02:00:00      NaN NaN            0.0                0
7  2025-06-08 02:10:00      NaN NaN            0.0                0
8  2025-06-08 02:20:00      NaN NaN            0.0                0
9  2025-06-08 02:30:00      NaN NaN            0.0                0
10 2025-06-08 02:40:00      NaN NaN            0.0                0
11 2025-06-08 02:50:00      NaN NaN            0.0                0
12 2025-06-08 03:00:00      NaN NaN            0.0                0
13 2025-06-08 03:10:00      NaN NaN            0

,timestamp,internal_air_temp_c,internal_radiation,ET0,internal_rh_pct,pred_internal_air_temp_c,pred_internal_radiation,pred_ET0,pred_internal_rh_pct,run,EC (ms),PH,Irrigation [ml],Fertilization Type,fertilizer_type,irrigation_ml
14956,2025-09-20 23:00:00,24.186035,0.0,0.000000,80.939095,24.573928,0.181958,0.000000,79.862971,103,0.68,6.8,NaN,NaN,0,0.0
14957,2025-09-20 23:10:00,23.259890,0.0,0.000000,90.562225,23.991317,0.388302,0.000000,85.325704,103,0.68,6.8,NaN,NaN,0,0.0
14958,2025-09-20 23:20:00,23.165852,0.0,0.000000,85.684230,23.751594,0.495577,0.000000,81.510711,103,0.68,6.8,NaN,NaN,0,0.0
14959,2025-09-20 23:30:00,23.148990,0.0,0.000000,84.351010,23.902163,0.000000,0.000000,81.533009,103,0.68,6.8,NaN,NaN,0,0.0
14960,2025-09-20 23:40:00,22.935820,0.0,0.000000,85.534430,23.499156,0.504540,0.000000,80.484855,103,0.68,6.8,NaN,NaN,0,0.0
14961,2025-09-20 23:50:00,22.579453,0.0,0.000000,85.171510,23.166633,0.193107,0.000000,80.033749,103,0.68,6.8,NaN,NaN,0,0.0
14962,2025-09-21 00:00:00,22.558565,0.0,0.000000,83.932280,22.904566,0.659156,0.000000,82.931993,103,0.68,6.8,NaN,NaN,0,0.0
14963,2025-09-21 00:10:00,22.410704,0.0,0.000000,81.611330,22.638827,0.651906,0.000000,81.862670,103,0.68,6.8,NaN,NaN,0,0.0
14964,2025-09-21 00:20:00,22.489520,0.0,0.000262,81.793520,22.656570,0.000000,0.000624,82.819506,103,0.68,6.8,NaN,NaN,0,0.0
14965,2025-09-21 00:30:00,21.866858,0.0,0.000825,88.338100,22.229888,0.000000,0.000850,86.586738,103,0.68,6.8,NaN,NaN,0,0.0


In [18]:
# Save master dataset independently
master_greenhouse_ts.to_excel(
    "master_greenhouse_time_series.xlsx",
    index=False
)